In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='2'


In [5]:
import torchaudio
from pathlib import Path
import itertools

SAMPLE_RATE = 44100

TYPES_TRACK = ["drums.wav", "bass.wav", "other.wav"]
COMB_TRACK = list(itertools.combinations(TYPES_TRACK, 2))
print(COMB_TRACK)

[('drums.wav', 'bass.wav'), ('drums.wav', 'other.wav'), ('bass.wav', 'other.wav')]


In [47]:
dirs = Path(dataset_path).glob("*")
d1,d2 = random.choices(list(dirs), k=2)
print(d1, d2)

dataset/youtube-5m-loops-valid/DRELLER - Control (Official Video)_2 dataset/youtube-5m-loops-valid/Dua Lipa - Blow Your Mind (Mwah) (Official Video)_3


In [78]:
import torchaudio
import torchvision
from torch.utils.data import Dataset
from pathlib import Path
import numpy as np 
import functools
import random

class LoopComb(Dataset):
    def __init__(self, path):
        dirs = Path(path).glob("*")

        # positive pairs
        self.positives = [(os.path.join(d, t1), os.path.join(d, t2)) for t1, t2 in COMB_TRACK for d in dirs]
        self.positives = [ (t1, t2) for t1, t2 in self.positives if (os.path.exists(t1) and os.path.exists(t2))]
        print("positives #", len(self.positives))

        # random pairs
        dirs = Path(path).glob("*")
        dirnames = list(dirs)
        self.negatives = []
        for _ in range(int(len(self.positives)/len(COMB_TRACK))):
            d1, d2 = random.choices(dirnames, k=2)
            self.negatives.extend( [(os.path.join(d1, t1), os.path.join(d2, t2)) for t1, t2 in COMB_TRACK])
        self.negatives = [ (t1, t2) for t1, t2 in self.negatives if (os.path.exists(t1) and os.path.exists(t2))]
        print("negatives #", len(self.negatives))

        self.items = self.positives.copy()
        self.items.extend(self.negatives)

        self.labels = [1] * len(self.positives)
        self.labels.extend([0] * len(self.negatives))
        assert len(self.items) == len(self.labels)

        self.length = len(self.items)
        print("all samples #", len(self.items))
        self.transforms = torchvision.transforms.Compose([torchaudio.transforms.Spectrogram()])#,torchaudio.transforms.AmplitudeToDB()])

    @functools.lru_cache(maxsize=500) # least recently used cache - 一番古く使われたものから消していく
    def __getitem__(self, index):
        (filepath1, filepath2), label = self.items[index], self.labels[index]
        tensor1 = self.convert_to_tensor(filepath1)
        tensor2 = self.convert_to_tensor(filepath2)
        return (tensor1, tensor2), label

    def __len__(self):
        return self.length

    def convert_to_tensor(self,filepath):
        audio_tensor, sr = torchaudio.load(filepath)
        audio_tensor = torchaudio.transforms.Resample(orig_freq=sr, new_freq=SAMPLE_RATE)(audio_tensor)
        return self.transforms(audio_tensor)

In [79]:
dataset_path = './dataset/youtube-5m-loops-valid'
dataset = LoopComb(dataset_path)

positives # 7169
negatives # 7167
all samples # 14336


In [80]:
dataset[0]

((tensor([[[2.1586e+00, 2.7074e-01, 1.7595e+00,  ..., 2.8635e-04,
            7.6017e-02, 1.8681e-01],
           [2.3474e+00, 3.6333e+00, 2.9481e+01,  ..., 4.1482e-02,
            3.5880e-02, 2.1071e-01],
           [4.8347e+00, 1.4895e+01, 1.0551e+02,  ..., 1.0192e-02,
            1.5693e-02, 7.0059e-02],
           ...,
           [8.8535e-10, 1.2337e-09, 3.0354e-08,  ..., 2.9494e-08,
            1.1952e-08, 1.1288e-08],
           [9.3206e-09, 1.7663e-09, 8.0608e-09,  ..., 7.3874e-08,
            5.3166e-09, 3.5547e-11],
           [2.7417e-08, 9.7663e-09, 3.6673e-09,  ..., 1.0210e-07,
            4.4345e-08, 2.0200e-08]],
  
          [[2.1080e+00, 2.6136e-01, 1.7178e+00,  ..., 3.1621e-04,
            7.7411e-02, 1.8923e-01],
           [2.3188e+00, 3.6125e+00, 2.9530e+01,  ..., 4.2025e-02,
            3.6707e-02, 2.1256e-01],
           [4.8907e+00, 1.5020e+01, 1.0565e+02,  ..., 1.0280e-02,
            1.5373e-02, 7.0507e-02],
           ...,
           [2.9341e-09, 1.2493e-09, 4